In [ ]:
!pip3 install mediacloud
!pip3 install python-dotenv


In [1]:
from dotenv import load_dotenv
import os, mediacloud.api
from IPython.display import JSON
load_dotenv()  # load config from .env file


True

In [2]:
from IPython.display import JSON


First we call the API:

In [3]:
import mediacloud.api
mc = mediacloud.api.MediaCloud('439141f9647ae800fa099bf77e4297ea28fe3f2bbc2d64b1f2163ad52fd8fef2')
mediacloud.__version__

'3.12.5'

In [4]:
mc.stats() #gives some info about Media Cloud

{'active_crawled_feeds': 161428,
 'active_crawled_media': 57895,
 'daily_downloads': 1113277,
 'daily_stories': 642103,
 'mediacloud_stats_id': 9,
 'stats_date': '2022-03-27',
 'total_downloads': 0,
 'total_sentences': 0,
 'total_stories': 0}

Measuring attention

In [6]:
my_query = '"voter ID" OR "voter identification" OR "voting ID" OR "ID law" and media_id:1 AND tags_id_media:34412234'
mc.storyCount(my_query)

{'count': 387}

In [10]:
my_query = '"voter ID" OR "voter identification" OR "voting ID" OR "ID law" OR "ID laws" and media_id:1092'
mc.storyCount(my_query)

{'count': 225}

In [11]:
import datetime
start_date = datetime.date(2013,1,1)
end_date = datetime.date(2021,1,1)
date_range = mc.dates_as_query_clause(start_date, end_date) # default is start & end inclusive
mc.storyCount(my_query, date_range)

{'count': 92}

In [12]:
# you can see this over time by using the `split` argument
# this defaults to results by day, but you can pass a split_period in (day, week, month, or year)
results = mc.storyCount(my_query, date_range, split=True, split_period='month')
JSON(results)

<IPython.core.display.JSON object>

In [14]:
relevant_stories = mc.storyCount(my_query, date_range)
total_stories = mc.storyCount('media_id:2', date_range)
source_ratio = relevant_stories['count'] / total_stories['count']
'{:.2%} of 2019-2022 Washington Post stories are about "voter ID"'.format(source_ratio)

'0.01% of 2019-2022 Washington Post stories are about "voter ID"'

Code below to retrieve stories from US national news collection

In [5]:
# check in our collection of country-level US National media sources
us_query = '"voter ID" or "voter identification" and tags_id_media:34412234'
mc.storyCount(us_query, date_range)

NameError: name 'date_range_2019' is not defined

In [4]:
# lets normalize this attention in the same way as we did previsouly
relevant_stories = mc.storyCount(us_query, date_range_2019)
total_stories = mc.storyCount('tags_id_media:34412234 AND media_id:2', date_range_2019)
us_country_ratio = relevant_stories['count'] / total_stories['count']
'{:.2%} of stories from national-level US media sources in 2019 mentioned "climate change"'.format(us_country_ratio)

NameError: name 'us_query' is not defined

In [28]:
# now we can compare this to the source-level coverage
coverage_ratio = source_ratio / us_country_ratio
'"climate change" received {:.2} times as much coverage in WashPo than you might expect based on other US national papers'.format(coverage_ratio)

'"climate change" received 2.6 times as much coverage in WashPo than you might expect based on other US national papers'

Listing Stories


In [16]:
# grab a random set of stories about this issue (defaults to 20 stories)
# note that this seed is fixed, so it will return the same set of random stories each call you make
stories = mc.storyList(my_query, date_range, sort=mc.SORT_RANDOM)
JSON(stories)

<IPython.core.display.JSON object>

In [17]:
def all_matching_stories(mc_client, q, fq):
    """
    Return all the stories matching a query within Media Cloud. Page through the results automatically.
    :param mc_client: a `mediacloud.api.MediaCloud` object instantiated with your API key already
    :param q: your boolean query
    :param fq: your date range query
    :return: a list of media cloud story items
    """
    last_id = 0
    more_stories = True
    stories = []
    while more_stories:
        page = mc_client.storyList(q, fq, last_processed_stories_id=last_id, rows=500, sort='processed_stories_id')
        print("  got one page with {} stories".format(len(page)))
        if len(page) == 0:
            more_stories = False
        else:
            stories += page
            last_id = page[-1]['processed_stories_id']
    return stories

In [19]:
# let's fetch all the stories matching our query (this can take a few minutes)
jan_2020 = mc.dates_as_query_clause(datetime.date(2013,1,1), datetime.date(2021,1,1))
all_stories = all_matching_stories(mc, my_query, jan_2020)
len(all_stories)

  got one page with 56 stories
  got one page with 0 stories


56

In [ ]:
for s in all_stories:
    print(s)

In [21]:
# first flatten things a little bit to make writing a CSV easier
import mediacloud.tags
for s in all_stories:
    # see the "language" notebook for more details on themes
    theme_tag_names = ','.join([t['tag'] for t in s['story_tags'] if t['tag_sets_id'] == mediacloud.tags.TAG_SET_NYT_THEMES])
    s['themes'] = theme_tag_names
# now write the CSV
import csv
fieldnames = ['stories_id', 'publish_date', 'title', 'url', 'language', 'ap_syndicated', 'themes', 'media_id', 'media_name', 'media_url']
with open('story-list.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
    writer.writeheader()
    for s in all_stories:
        writer.writerow(s)

In [22]:
# and let's make sure it worked by checking out by loading it up as a pandas DataFrame
import pandas
pandas.read_csv('story-list.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 3691: invalid start byte

### Media Cloud: Studying Language

In [ ]:
results = mc.wordCount(my_query, date_range_2019)
results[:10]

In [38]:
# check how many stories were about this issue
story_count = mc.storyCount(my_query)['count']
story_count

387

In [39]:
doc_term_matrix = mc.storyWordMatrix(my_query, date_range_2019, rows=story_count)


In [ ]:
doc_term_matrix

In [65]:
json = JSON(doc_term_matrix)
jsonstr = str(json)


In [62]:
import json
import urllib
import pandas as pd
import nltk
from nltk.corpus import state_union
from sklearn.datasets import fetch_20newsgroups 

In [71]:
df4 = pd.read_json('data.json') 

ValueError: Expected object or value

In [69]:
# Convert JSON to DataFrame Using read_json()
df = pd.read_json(jsonstr, orient ='index')

ValueError: Expected object or value

In [41]:
# see how many documents a particular term is used in
term_index = 2
docs_including_term = [stories_id 
                       for stories_id, term_lookup in doc_term_matrix['word_matrix'].items()
                       if str(term_index) in term_lookup.keys()]
'{} docs include the term "{}"'.format(len(docs_including_term), doc_term_matrix['word_list'][term_index][1])

'3 docs include the term "afraid"'

In [58]:
import pandas as pd
import json